In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import os
import librosa
import librosa.display
import IPython.display as ipd
from itertools import cycle
from tqdm import tqdm

In [2]:
class Configuration:

  class Paths:
    train_audio_folders = '/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_audio'
    train_audio_files='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_audio/*/*.ogg'
    unlabeld_soundspaces='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/unlabeled_soundscapes'
    taxonomy_birds = '/content/drive/MyDrive/Kaggle/Bird_CLEF2024/eBird_Taxonomy_v2021.csv'
    meta_data='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_metadata.csv'
    sample_submission='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/sample_submission.csv'


  class ClassInfo:
    class_names=sorted(os.listdir('/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_audio'))
    num_classes=len(class_names)
    labels=list(range(num_classes))
    labels_with_names = dict(zip(labels,class_names))

config=Configuration()

In [3]:
labels_names=config.ClassInfo.labels_with_names
meta_data=pd.read_csv(config.Paths.meta_data)
meta_data['filepath'] = config.Paths.train_audio_folders+'/'+meta_data.filename

In [8]:
# Code for extracting the audio signal and sample rate of single example of each bird species

birds=meta_data['primary_label'].unique()
df= meta_data.set_index('primary_label')
bird_species=[]
audio_signal=[]
sample_rate=[]
audio_data={}
for bird in tqdm(birds, desc='Loading audio signal and sample rate'):

  path = df.loc[bird].iloc[0,:]['filepath']
  audio, sr = librosa.load(path)
  bird_species.append(bird)
  audio_signal.append(audio)
  sample_rate.append(sr)

audio_data={'bird_species':bird_species,'audio_signal':audio_signal,'sample_rate':sample_rate}


Loading audio signal and sample rate: 100%|██████████| 182/182 [01:24<00:00,  2.16it/s]


In [ ]:
# Code for extracting the audio signal and sample rate of all the examples of each bird species

birds=meta_data['primary_label'].unique()
df= meta_data.set_index('primary_label')
bird_species=[]
audio_signal=[]
sample_rate=[]
audio_data={}
audio, sr = 0,0
for bird in birds:
  if bird.startswith('c'):

    for i in tqdm(range(df.loc[bird].shape[0]),desc=f'Loading audio signal and sample rate for {bird}'):
      path = df.loc[bird].iloc[0,:]['filepath']
      audio, sr = librosa.load(path)
      bird_species.append(bird)
      audio_signal.append(audio)
      sample_rate.append(sr)

    globals()[f'audio_data_{bird[0]}']={'bird_species':bird_species,'audio_signal':audio_signal,'sample_rate':sample_rate}
  else:
    continue

In [10]:
import sys

# Calculate the total memory usage of audio_data_c
total_size = sys.getsizeof(audio_data)
for key, value in audio_data.items():
    total_size += sys.getsizeof(key) + sys.getsizeof(value)
    if isinstance(value, list):
        for item in value:
            total_size += sys.getsizeof(item)
# Convert total memory size to gigabytes
total_size_gb = total_size / (1024 ** 3)

print("Total memory usage of audio_data_c:", total_size_gb, "GB")




Total memory usage of audio_data_c: 0.17704520467668772 GB
